#### Importa bibliotecas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

#### Prepara Dataset

In [4]:
df = pd.read_csv('dengue_sinan.csv', low_memory=False)
print(df.head())

   NU_NOTIFIC  TP_NOT ID_AGRAVO  DT_NOTIFIC  SEM_NOT  NU_ANO  SG_UF_NOT  \
0         158       2       A90  2016-03-05   201609    2016         29   
1         298       2       A90  2016-02-15   201607    2016         29   
2        5082       2       A90  2016-03-25   201612    2016         29   
3      111262       2       A90  2016-03-24   201612    2016         29   
4         166       2       A90  2016-03-13   201611    2016         29   

   ID_MUNICIP  ID_REGIONA  ID_UNIDADE  ... DT_TRANSRM  DT_TRANSRS  DT_TRANSSE  \
0      292210      1381.0   2498731.0  ...        NaN         NaN         NaN   
1      293290      1385.0   3280969.0  ...        NaN         NaN         NaN   
2      293250      1385.0   2800527.0  ...        NaN         NaN         NaN   
3      291360      1385.0   2706628.0  ...        NaN         NaN         NaN   
4      292210      1381.0   2498731.0  ...        NaN         NaN         NaN   

   NU_LOTE_V NU_LOTE_H  CS_FLXRET  FLXRECEBI  IDENT_MICR  MIGR

#### Faz pré-processamento

In [6]:
# Selecionando colunas que não possuem valores númericos para avaliar se devemos dummizar, converter ou eliminar

non_numeric_columns = df.select_dtypes(exclude=['number']).columns.tolist()

non_numeric_columns

['ID_AGRAVO',
 'DT_NOTIFIC',
 'DT_SIN_PRI',
 'CS_SEXO',
 'NM_BAIRRO',
 'NM_REFEREN',
 'DT_INVEST',
 'ID_OCUPA_N',
 'DT_CHIK_S1',
 'DT_CHIK_S2',
 'DT_PRNT',
 'DT_SORO',
 'DT_NS1',
 'DT_VIRAL',
 'DT_PCR',
 'DT_INTERNA',
 'NOBAIINF',
 'DT_OBITO',
 'DT_ENCERRA',
 'DT_ALRM',
 'DT_GRAV',
 'DS_OBS',
 'DT_DIGITA',
 'DT_TRANSSM',
 'ID_CNS_SUS_HASHED']

In [7]:
# Garantir a conversao das datas para numeric (precisamos delas numeric para posteriormentes normaliza-las com o scaler)
date_columns = [label for label in non_numeric_columns if "DT_" in label]
for date_column in date_columns:
  df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
  df[date_column] = pd.to_numeric(df[date_column])

In [8]:
# Colunas não numéricas restantes
non_numeric_without_date = set(non_numeric_columns) - set(date_columns)
non_numeric_without_date

{'CS_SEXO',
 'DS_OBS',
 'ID_AGRAVO',
 'ID_CNS_SUS_HASHED',
 'ID_OCUPA_N',
 'NM_BAIRRO',
 'NM_REFEREN',
 'NOBAIINF'}

In [9]:
# Decidimos remover as seguintes colunas pelos seguintes motivos:
# ID_OCUPA_N = Indica somente o tipo de ocupação (trabalho) do paciente, pode influenciar mas possui muitos valores nulos ou discrepantes (dummização ineficaz)
# ID_CNS_SUS_HASHED = Um hash identificador, dummização não surtiria efeito
# NM_BAIRRO = Nome do bairro, pode auxiliar a identificar focos de dengue e relação moradia x zona de infestação, porém possui muitos valores discrepantes
# e a dummização não seria eficaz
# NM_REFEREN = Está relacionado a referencia de endereço, dummização ineficiente
# DS_OBS = Inforações adicionais referentes ao caso, não é possível dummizar e os modelos não conseguiriam classificar com base nessa informação
# NOBAIINF = Indica o nome do bairro da possível infecção, pode ser útil mas difícil dummizar, além de que possui outra coluna com código referente

df.drop(columns=['ID_OCUPA_N', 'ID_CNS_SUS_HASHED', 'NM_BAIRRO', 'NM_REFEREN', 'DS_OBS', 'NOBAIINF'], inplace=True)

# Vamos aproveitar também dummizar as colunas não númericas restantes e observar quais vale a pena manter

df = pd.get_dummies(df, columns=['CS_SEXO'], prefix='SEXO', dtype=float)
df = pd.get_dummies(df, columns=['ID_AGRAVO'],prefix='AGRAVO', dtype=float)

df

,NU_NOTIFIC,TP_NOT,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,...,NU_LOTE_V,NU_LOTE_H,CS_FLXRET,FLXRECEBI,IDENT_MICR,MIGRADO_W,SEXO_F,SEXO_I,SEXO_M,AGRAVO_A90
0,158,2,1457136000000000000,201609,2016,29,292210,1381.0,2498731.0,1456876800000000000,...,NaN,NaN,0.0,NaN,4.0,NaN,0.0,0.0,1.0,1.0
1,298,2,1455494400000000000,201607,2016,29,293290,1385.0,3280969.0,1455408000000000000,...,NaN,NaN,0.0,NaN,4.0,NaN,0.0,0.0,1.0,1.0
2,5082,2,1458864000000000000,201612,2016,29,293250,1385.0,2800527.0,1458777600000000000,...,NaN,NaN,1.0,NaN,4.0,NaN,1.0,0.0,0.0,1.0
3,111262,2,1458777600000000000,201612,2016,29,291360,1385.0,2706628.0,1458691200000000000,...,NaN,NaN,0.0,NaN,4.0,NaN,1.0,0.0,0.0,1.0
4,166,2,1457827200000000000,201611,2016,29,292210,1381.0,2498731.0,1457740800000000000,...,NaN,NaN,0.0,NaN,4.0,NaN,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620206,7061,2,1708473600000000000,202408,2024,29,292300,1388.0,2506254.0,1708473600000000000,...,NaN,NaN,0.0,NaN,4.0,NaN,0.0,0.0,1.0,1.0
620207,1257,2,1709510400000000000,202410,2024,29,291980,1398.0,7175973.0,1709078400000000000,...,NaN,NaN,0.0,NaN,4.0,NaN,0.0,0.0,1.0,1.0
620208,372024,2,1706227200000000000,202404,2024,29,292303,1381.0,6540007.0,1705795200000000000,...,NaN,NaN,0.0,NaN,4.0,NaN,1.0,0.0,0.0,1.0
620209,128,2,1708300800000000000,202408,2024,29,291400,1381.0,2824558.0,1708214400000000000,...,NaN,NaN,1.0,NaN,4.0,NaN,1.0,0.0,0.0,1.0


In [10]:
# Após observar que ID_AGRAVO possui somente 1 coluna DUMMIE (A90), removemos-a, já que não fará diferença para a predição

df.drop(columns=['AGRAVO_A90'], inplace=True)
df

,NU_NOTIFIC,TP_NOT,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,...,DT_TRANSSE,NU_LOTE_V,NU_LOTE_H,CS_FLXRET,FLXRECEBI,IDENT_MICR,MIGRADO_W,SEXO_F,SEXO_I,SEXO_M
0,158,2,1457136000000000000,201609,2016,29,292210,1381.0,2498731.0,1456876800000000000,...,NaN,NaN,NaN,0.0,NaN,4.0,NaN,0.0,0.0,1.0
1,298,2,1455494400000000000,201607,2016,29,293290,1385.0,3280969.0,1455408000000000000,...,NaN,NaN,NaN,0.0,NaN,4.0,NaN,0.0,0.0,1.0
2,5082,2,1458864000000000000,201612,2016,29,293250,1385.0,2800527.0,1458777600000000000,...,NaN,NaN,NaN,1.0,NaN,4.0,NaN,1.0,0.0,0.0
3,111262,2,1458777600000000000,201612,2016,29,291360,1385.0,2706628.0,1458691200000000000,...,NaN,NaN,NaN,0.0,NaN,4.0,NaN,1.0,0.0,0.0
4,166,2,1457827200000000000,201611,2016,29,292210,1381.0,2498731.0,1457740800000000000,...,NaN,NaN,NaN,0.0,NaN,4.0,NaN,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620206,7061,2,1708473600000000000,202408,2024,29,292300,1388.0,2506254.0,1708473600000000000,...,NaN,NaN,NaN,0.0,NaN,4.0,NaN,0.0,0.0,1.0
620207,1257,2,1709510400000000000,202410,2024,29,291980,1398.0,7175973.0,1709078400000000000,...,NaN,NaN,NaN,0.0,NaN,4.0,NaN,0.0,0.0,1.0
620208,372024,2,1706227200000000000,202404,2024,29,292303,1381.0,6540007.0,1705795200000000000,...,NaN,NaN,NaN,0.0,NaN,4.0,NaN,1.0,0.0,0.0
620209,128,2,1708300800000000000,202408,2024,29,291400,1381.0,2824558.0,1708214400000000000,...,NaN,NaN,NaN,1.0,NaN,4.0,NaN,1.0,0.0,0.0


In [11]:
# Ao analisar o dicionário, é possível observar que CLASSI_FIN é a coluna que define a classificação do caso.
# Transformaremos (10, 11 e 12) em 1.0 para Dengue e demais para 0.0 Não Dengue, conforme indicado no mesmo

# Antes pegaremos somente as linhas em que CLASSI_FIN não é nulo
df.dropna(subset=['CLASSI_FIN'], inplace=True)

df['CLASSI_FIN_BINARIO'] = df['CLASSI_FIN'].apply(lambda x: 1.0 if x in [10.0, 11.0 ,12.0] else 0.0)
df.drop(columns=['CLASSI_FIN'], inplace=True)

In [12]:
# Limpamos as colunas que apresentam somente valores nulos
null_columns = df.columns[df.isnull().all()].tolist()
df.drop(columns=null_columns, inplace=True)

In [13]:
# Buscamos por colunas que possuem valores constantes
const_cols = df.columns[df.nunique(dropna=False) == 1]
const_cols

# Observamos que somente ID_AGRAVO tinha valores constantes, então já eliminamos a única coluna constante

Index([], dtype='object')

In [14]:
# Definimos quais colunas desejamos obrigatoriamente em nosso dataset (manteremos pelo menos sinais clínicos e doenças pré-existentes)
columns_to_keep = {'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO',
                   'DOR_RETRO', 'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 'ACIDO_PEPT', 'AUTO_IMUNE'}

In [15]:
# Observamos agora a correlação de CLASSI_FIN_BINARIO para as demais colunas
# Pegamos somente as colunas que não tenham uma alta correlação com CLASSI_FIN_BINARIO (positiva ou negativa)
# conforme a regra do polegar do artigo https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3576830/
corr = df.corr()['CLASSI_FIN_BINARIO']
not_high_corr_columns = set(df.columns[abs(corr) < 0.7].tolist())

In [16]:
# Agora juntamos as colunas que queremos manter (obrigatoriamente) com as filtradas por correlação e obtemos as colunas
selected_columns = list(columns_to_keep.union(not_high_corr_columns))
# Readicionamos CLASSI_FIN_BINARIO (já que a correlação dela para com ela mesma é 1)
selected_columns.append('CLASSI_FIN_BINARIO')
df = df[selected_columns]
df.columns

Index(['DIABETES', 'DT_PRNT', 'ACIDO_PEPT', 'LACO', 'ALRM_SANG', 'GRAV_ORGAO',
       'ID_MUNICIP', 'MANI_HEMOR', 'ARTRALGIA', 'SEM_NOT',
       ...
       'HEMATOLOG', 'CRITERIO', 'DT_CHIK_S1', 'DT_NS1', 'GRAV_CONSC',
       'COUFINF', 'DT_SORO', 'ALRM_HEPAT', 'MIALGIA', 'CLASSI_FIN_BINARIO'],
      dtype='object', length=116)

In [17]:
# Ainda temos um problema, valores nulos não são aceitos por modelos como KNN, porém injetar nulo como 0 pode ser perigoso
# já que não ter um valor não significa necessariamente que ele está ou não presente (ele pode estar tanto presente quanto não presente)
# portanto, por não haver números negativos dentre os escolhidos, padronizaremos -1.0 como o valor referente a nulo em todas as colunas
# um adendo, é que todas as colunas restantes são do tipo numérico, o que implica em -1.0 ser um valor aceito

df.fillna(-1.0, inplace=True)
df

C:\Users\leand\AppData\Local\Temp\ipykernel_13960\4180065339.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(-1.0, inplace=True)


,DIABETES,DT_PRNT,ACIDO_PEPT,LACO,ALRM_SANG,GRAV_ORGAO,ID_MUNICIP,MANI_HEMOR,ARTRALGIA,SEM_NOT,...,HEMATOLOG,CRITERIO,DT_CHIK_S1,DT_NS1,GRAV_CONSC,COUFINF,DT_SORO,ALRM_HEPAT,MIALGIA,CLASSI_FIN_BINARIO
0,-1.0,-9223372036854775808,-1.0,-1.0,-1.0,-1.0,292210,-1.0,-1.0,201609,...,-1.0,-1.0,-9223372036854775808,-9223372036854775808,-1.0,-1.0,-9223372036854775808,-1.0,-1.0,0.0
1,-1.0,-9223372036854775808,-1.0,-1.0,-1.0,-1.0,293290,-1.0,-1.0,201607,...,-1.0,-1.0,-9223372036854775808,-9223372036854775808,-1.0,-1.0,-9223372036854775808,-1.0,-1.0,0.0
2,2.0,-9223372036854775808,2.0,2.0,-1.0,-1.0,293250,-1.0,1.0,201612,...,2.0,2.0,-9223372036854775808,-9223372036854775808,-1.0,29.0,-9223372036854775808,-1.0,1.0,1.0
3,-1.0,-9223372036854775808,-1.0,-1.0,-1.0,-1.0,291360,-1.0,-1.0,201612,...,-1.0,2.0,-9223372036854775808,-9223372036854775808,-1.0,-1.0,-9223372036854775808,-1.0,-1.0,1.0
4,-1.0,-9223372036854775808,-1.0,-1.0,-1.0,-1.0,292210,-1.0,-1.0,201611,...,-1.0,-1.0,-9223372036854775808,-9223372036854775808,-1.0,-1.0,-9223372036854775808,-1.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619929,-1.0,-9223372036854775808,-1.0,-1.0,-1.0,-1.0,292710,-1.0,-1.0,202404,...,-1.0,-1.0,-9223372036854775808,-9223372036854775808,-1.0,-1.0,-9223372036854775808,-1.0,-1.0,0.0
619967,-1.0,-9223372036854775808,-1.0,-1.0,-1.0,-1.0,290320,-1.0,-1.0,202402,...,-1.0,-1.0,-9223372036854775808,-9223372036854775808,-1.0,-1.0,-9223372036854775808,-1.0,-1.0,0.0
620081,-1.0,-9223372036854775808,-1.0,-1.0,-1.0,-1.0,292740,-1.0,-1.0,202406,...,-1.0,-1.0,-9223372036854775808,-9223372036854775808,-1.0,-1.0,-9223372036854775808,-1.0,-1.0,0.0
620100,-1.0,-9223372036854775808,-1.0,-1.0,-1.0,-1.0,293050,-1.0,-1.0,202405,...,-1.0,-1.0,-9223372036854775808,-9223372036854775808,-1.0,-1.0,-9223372036854775808,-1.0,-1.0,0.0


In [18]:
# Agora vamos normalizar o dataset baseado minmax
from sklearn.preprocessing import MinMaxScaler

# criando um min max scaler
scaler = MinMaxScaler()

# transformado os dados
normalized_data = scaler.fit_transform(df)

normalized_df = pd.DataFrame(normalized_data, columns=df.columns)
normalized_df

,DIABETES,DT_PRNT,ACIDO_PEPT,LACO,ALRM_SANG,GRAV_ORGAO,ID_MUNICIP,MANI_HEMOR,ARTRALGIA,SEM_NOT,...,HEMATOLOG,CRITERIO,DT_CHIK_S1,DT_NS1,GRAV_CONSC,COUFINF,DT_SORO,ALRM_HEPAT,MIALGIA,CLASSI_FIN_BINARIO
0,0.0,0.0,0.0,0.0,0.0,0.0,0.433818,0.0,0.000000,0.066125,...,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.436389,0.0,0.000000,0.063805,...,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,1.0,0.0,1.0,1.0,0.0,0.0,0.436294,0.0,0.666667,0.069606,...,1.0,0.75,0.0,0.0,0.0,0.555556,0.0,0.0,0.666667,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.431794,0.0,0.000000,0.069606,...,0.0,0.75,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.433818,0.0,0.000000,0.068445,...,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549792,0.0,0.0,0.0,0.0,0.0,0.0,0.435008,0.0,0.000000,0.988399,...,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
549793,0.0,0.0,0.0,0.0,0.0,0.0,0.429318,0.0,0.000000,0.986079,...,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
549794,0.0,0.0,0.0,0.0,0.0,0.0,0.435079,0.0,0.000000,0.990719,...,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
549795,0.0,0.0,0.0,0.0,0.0,0.0,0.435818,0.0,0.000000,0.989559,...,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [19]:
# Separamos em treino e validação e estratificamos com base em Y para manter a proporção

from sklearn.model_selection import train_test_split

X = normalized_df.drop(['CLASSI_FIN_BINARIO'], axis=1)
Y = normalized_df['CLASSI_FIN_BINARIO']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=45, stratify=Y)

In [20]:
# Confirmamos a estratificação ao ver a proporção no conjunto de treino e validação
import numpy as np

print(np.unique(Y_train, return_counts=True))
print(np.unique(Y_val, return_counts=True))

(array([0., 1.]), array([262916, 176921], dtype=int64))
(array([0., 1.]), array([65729, 44231], dtype=int64))


In [21]:
#exporta o conjunto de teste
df_teste = X_val.copy()
df_teste['CLASSI_FIN_BINARIO'] = Y_val
df_teste.to_csv('dataset2_teste.csv', index=False)